<a href="https://colab.research.google.com/github/giacomo-monni/ML_Project/blob/main/ML_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

Importing necessary Python packages to process the data.

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Loading the Dataset
The Malicious URL's Dataset is composed of:
- Benign/safe URLs
- Defacement URLs
- Phishing URLs
- Malware URLs

The dataset (downloaded from https://www.kaggle.com/datasets/naveenbhadouria/malicious/code) has been stored into my Google Drive.

In order to obtain the file, the PyDrive2 package has been used: it allows to authenticate to Google Drive and obtain the file starting from its ID.



In [3]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
# File ID on Google Drive
file_id = '1iBTfKOyuwL-S-bkLKkjmCgdAih9XCxSR'

# Obtaining the file with the specified ID
# and downloading the file to a local path
drive_file = drive.CreateFile({'id': file_id})
drive_file.GetContentFile('malicious_phish1.csv')

# Reading the .csv file
df = pd.read_csv('malicious_phish1.csv')
print(df.head())

                                                 url        type
0                                   br-icloud.com.br    phishing
1                mp3raid.com/music/krizz_kaliko.html      benign
2                    bopsecrets.org/rexroth/cr/1.htm      benign
3  http://www.garage-pirenne.be/index.php?option=...  defacement
4  http://adventure-nicaragua.net/index.php?optio...  defacement
